# Universal Automated Tutor

## 0: Data Import + Cleaning

In [6]:
import pandas as pd
import numpy as np
import random

In [7]:
# Read in the CSVs, select variables, combine
raw_data04 = pd.read_csv("2004-WPI-Assistments-Math.csv",low_memory=False)
raw_data056 = pd.read_csv("2005-06-WPI-Assistments-Math.csv",low_memory=False)
raw_data056["problem_id"] = raw_data056["problem_id"] + max(raw_data04.problem_id)
raw_data = pd.concat([raw_data056,raw_data04])
data = raw_data[["stud_id","duration","student_response_type","problem_id","step","attempt_num",
             "last_attempt","outcome","input","feedback"]]
data.reset_index(drop=True, inplace=True)

# Create binary variables for hint, correct, incorrect
data.loc[:,"hint"] = np.where(data["outcome"] == "HINT",1,0)
data.loc[:,"correct"] = np.where(data["outcome"] == "CORRECT",1,0)
data.loc[:,"incorrect"] = np.where(data["outcome"] == "INCORRECT",1,0)

# Filter + clean data
data = data[data["duration"] != '.']
data["duration"] = data["duration"].astype(np.float64)

C:\Users\C22Evan.Marrone\Anaconda3\lib\site-packages\pandas\core\indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
C:\Users\C22Evan.Marrone\Anaconda3\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [8]:
correct = data[data['outcome'] == "CORRECT"]
uniqueCorrect = correct.drop_duplicates(subset=['problem_id'])
pairs = uniqueCorrect[["problem_id","input"]]
data = data[data.problem_id.isin(pairs.problem_id)]

## 1: Distribution Creation

In [9]:
# Docstring Example

''' Summary or Description of the Function

    Parameters:
    argument1 (int): Description of arg1

    Returns:
    int: Returning value
'''

' Summary or Description of the Function\n\n    Parameters:\n    argument1 (int): Description of arg1\n\n    Returns:\n    int: Returning value\n'

In [10]:
def chi2lower(x):
    ''' Calculates the chi-squared distribution lower bound

    Parameters:
    x (float64): A # of incorrect, # of hints, or a duration

    Returns:
    float64: Returns the lower bound of the chi-squared distribution
    '''
    return np.mean(x) - np.std(x)/4


def chi2upper(x):
    ''' Calculates the chi-squared distribution upper bound

    Parameters:
    x (float64): A # of incorrect, # of hints, or a duration

    Returns:
    float64: Returns the upper bound of the chi-squared distribution
    '''
    return np.mean(x) + np.std(x)/2

In [11]:
# group the problems and students together to get the sum of each variable for each question
df = data[["problem_id", "duration", "hint", "stud_id","incorrect"]].groupby(["problem_id","stud_id"]).sum()
problemDistsLower = df.groupby(["problem_id"]).agg(chi2lower)
problemDistsUpper = df.groupby(["problem_id"]).agg(chi2upper)
problemDistsLower.rename(columns={'duration': 'durationLower', 'hint': 'hintLower', 'incorrect': 'incorrectLower'}, inplace=True)
problemDistsUpper.rename(columns={'duration': 'durationUpper', 'hint': 'hintUpper', 'incorrect': 'incorrectUpper'}, inplace=True)
problemDists = pd.concat([problemDistsLower, problemDistsUpper], axis=1)

# Create given student's mean
studentEV = df.groupby(["stud_id"]).mean()

## 2: Student Class

In [28]:
class Student:
    def __init__(self, data):
        self.data = data
        self.means = 0
        self.problem = ""
        self.answer = ""

        
    def updateStudent(self, lastQ):
        self.data = self.data.append(lastQ)
        self.means = self.data.mean()
        self.duration = self.means.duration
        self.incorrect = self.means.incorrect
        self.hint = self.means.hint
        
    def nextQ(self, groupDf):
        # Determines a viable next question
        EVdur = self.duration
        EVinc = self.incorrect
        EVhint = self.hint
        viableProbs = problemDists[(EVdur < problemDists["durationUpper"])  & (EVhint < problemDists["hintUpper"]) & \
                                   (problemDists["incorrectLower"] > EVinc)]
        problemNum = random.choice(viableProbs.index.values.tolist())
        self.problem = groupDf[groupDf["problem_id"]==problemNum].iloc[0]["step"].split(':')[-1].split("?")[0] + "?"
        answer = pairs[pairs["problem_id"]==problemNum].iloc[0][1]
        if answer[0] in ["A","B","C","D"]:
            answer = answer[3:]
        self.answer = answer

## 3: Example Run Through

Pseudo Code
studentData = Pretest data
Pretest data will include entries with "step", "problem_id", "duration", "hint", "incorrect", "correct"
student = Student(studentData)

In [13]:
# Example
stud = "Stu_fe96fe63d83aa63c4ec667167fc7f1ce"
df = data[["step", "problem_id", "stud_id","duration", "hint", "incorrect", "correct"]]
newdf = df[df["stud_id"] == stud]
studentData = newdf.iloc[0:10,:]

studentData

,step,problem_id,stud_id,duration,hint,incorrect,correct
1266120,Step0:3761:Harry measured all but one angle of...,3761,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,90.0,1,0,0
1266121,Step1:3762:How many angles are there in a pent...,3761,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,37.0,0,1,0
1266122,Step1:3762:How many angles are there in a pent...,3761,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,4.0,0,0,1
1266123,Step0:3761:Harry measured all but one angle of...,3761,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,246.0,1,0,0
1266124,Step1:3762:How many angles are there in a pent...,3761,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,6.0,0,0,1
1266125,Step3:3764:What is the measure of the only ang...,3761,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,29.0,0,0,1
1266126,Step0:193:(0.2)(0.2)(0.2) is equal to which of...,193,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,37.0,0,0,1
1266127,Step0:113:Figure ABCD is translated so that th...,113,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,65.0,1,0,0
1266128,Step1:114:Look at the figure above. Let s find...,113,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,88.0,1,0,0
1266129,Step1:114:Look at the figure above. Let s find...,113,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,21.0,1,0,0


In [29]:
# Initializing a student
student = Student(studentData)

# questionData represents their next action from the GUI (we will be passed this)
questionData = newdf.iloc[11:13,:]
questionData

,step,problem_id,stud_id,duration,hint,incorrect,correct
1266131,Step1:114:Look at the figure above. Let s find...,113,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,21.0,1,0,0
1266132,Step1:114:Look at the figure above. Let s find...,113,Stu_fe96fe63d83aa63c4ec667167fc7f1ce,8.0,0,0,1


In [31]:
# Initializing a student
student = Student(studentData)

# questionData represents their next action from the GUI (we will be passed this)
questionData = newdf.iloc[11:13,:]

# If the person gets the problem right, then send the questionData to the student to update and choose next Q
if questionData.correct.isin([1]).any():
    student.updateStudent(questionData)
    student.nextQ(data)

# Question that is sent to the interface next:
student.problem

'Evaluate this expression if m = 5, n = -8, and p = -3. p - m + n?'

In [32]:
student.answer

'-16'

# What is left to finish model?

- Need to find correct answers
- Need to find images
- Need to figure out how to get through steps
- Need to format answer box to accept all close answers

In [ ]:
correct = data[data['outcome'] == "CORRECT"]
uniqueCorrect = correct.drop_duplicates(subset=['problem_id'])
pairs = uniqueCorrect[["problem_id","input"]]
pairs

In [ ]:
answer = pairs[pairs["problem_id"]==13450]["input"]

In [ ]:
pairs[pairs["problem_id"]==13450].iloc[0][1]

In [ ]:
data1 = data[data.problem_id.isin(pairs.problem_id)]
data1

In [ ]:
g = data[truth_series]
g

In [ ]:
df[df["problem_id"]==7].step

In [ ]:
import sys
from time import sleep

from PyQt6.uic import loadUi
from PyQt6.QtWidgets import QDialog, QApplication
from PyQt6 import QtWidgets


class QuestionScreen(QDialog):
    def __init__(self):
        super(QuestionScreen, self).__init__()
        loadUi("QuestionScreen.ui", self)
        self.QuestionLabel.setText(question)
        self.submit.clicked.connect(self.gotoresult)
        self.hintButton.clicked.connect(self.showhint)

    def gotoresult(self):
        questionanswer = "1"
        useranswer = self.answerBox.text()
        # print(useranswer)

        # Detect keystroke and remove empty ans text

        if useranswer == "":
            sleep(.1)
            self.emptyans.setText("Please enter an answer")
        elif useranswer == questionanswer:
            sleep(.1)
            correct = CorrectScreen()
            widget.addWidget(correct)
            widget.setCurrentIndex(widget.currentIndex() + 1)
        else:
            sleep(.1)
            wrong = WrongScreen()
            widget.addWidget(wrong)
            widget.setCurrentIndex(widget.currentIndex() + 1)

    def showhint(self):
        sleep(.1)
        hinttext = "Here is the hint for this question!"
        self.hintLabel.setText(hinttext)
    
    def showQuestion(self, question):
        sleep(.1)
        questiontext = question


class WrongScreen(QDialog):
    def __init__(self):
        super(WrongScreen, self).__init__()
        loadUi("IncorrectScreen_2.ui", self)
        self.submit.clicked.connect(self.returntoquestion)

    def returntoquestion(self):
        sleep(.1)
        question = QuestionScreen()
        widget.addWidget(question)
        widget.setCurrentIndex(widget.currentIndex() + 1)

class CorrectScreen(QDialog):
    def __init__(self):
        super(CorrectScreen, self).__init__()
        loadUi("CorrectScreen_2.ui", self)
        self.submit.clicked.connect(self.returntoquestion)

    def returntoquestion(self):
        sleep(.1)
        question = QuestionScreen()
        widget.addWidget(question)
        widget.setCurrentIndex(widget.currentIndex() + 1)



# main
app = QApplication(sys.argv)
question = QuestionScreen()
widget = QtWidgets.QStackedWidget()
widget.addWidget(question)
widget.setFixedHeight(300)
widget.setFixedWidth(400)
widget.show()
try:
    sys.exit(app.exec())
except:
    print("Exiting")
